In [6]:
!pip install pytest-mock

In [7]:
!pip install pytest pytest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [14]:
%%writefile market_order.py
import asyncio
from datetime import datetime

# Mock classes

class OrderType:
    Market = 'Market'



class Security:
    def __init__(self, symbol, price):
        self.symbol = symbol
        self.price = price



class Order:
    def __init__(self, symbol, quantity, time, tag='', properties=None):
        self.symbol = symbol
        self.quantity = quantity
        self.time = time
        self.tag = tag
        self.properties = properties

# main MarketOrder class

class MarketOrder(Order):
    def __init__(self, symbol, quantity, time, price=None, tag='', properties=None):
        super().__init__(symbol, quantity, time, tag, properties)
        self.price = price

    @property
    def type(self):
        return OrderType.Market

    async def get_value(self, security):

        return  self.quantity * security.price

    def clone(self):
        order = MarketOrder(self.symbol, self.quantity, self.time, self.price, self.tag, self.properties)
        return order




Overwriting market_order.py


In [21]:
%%writefile test_market_order.py
import pytest
from unittest.mock import AsyncMock
from datetime import datetime
from market_order import MarketOrder, Security

@pytest.mark.asyncio
async def test_market_order_get_value(mocker):

    mock_security = mocker.patch('market_order.Security')

    mock_security.symbol = "AAPL"
    mock_security.price = 150

    order = MarketOrder(symbol="AAPL", quantity=10, time=datetime.now())

    mock_get_value = mocker.patch.object(order, 'get_value', new_callable=AsyncMock)
    mock_get_value.return_value = 1500

    value = await order.get_value(mock_security)
    assert value == 1500


def test_market_order_clone(mocker):

    order = MarketOrder(symbol="AAPL", quantity=10, time=datetime.now())

    mock_response = AsyncMock()
    mocker.patch.object(order, "clone", return_value=mock_response)

    cloned_order = order.clone()
    assert cloned_order == mock_response


Overwriting test_market_order.py


In [22]:
!pytest test_market_order.py --asyncio-mode=auto

/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 2 items                                                                   